## Ideas for project:

### Create a "Star Power" metric based on Oscar nominations/wins:

#### Set up a loop to:
* Search each entry in df movies: 
    - (1) save "title_year" (need to change this to int), 
    - (2) save string within column actor_1_name, 
    - (3) search df oscars with conditional statement: if year is < title_year and name is the same, add 1 to counter for number of Oscars (or count the number of entries), 
    - (4) add new column to movies called "actor_1_oscar_nods" and enter the number from the counter
    - (5) repeat for actor_2_name and actor_3_name
* Note that this will count Oscar nominations and wins.  For wins only, set conditional statement to look for "True" under "winner" column

### Another metric of star power would be salary paid to the top actors/actresses:
* Here is a data set of the highest growwing actors of all time, but it's behind a 49 dollar pay wall: https://www.statista.com/statistics/655480/all-time-top-grossing-actors-box-office/
* 

### For release dates, there is a database of 45,000 movies on kaggle that includes full release date:
* https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings.csv
* Try joining with our data set?
* Should we look for dates that are big opening weekend (Memorial Day, Christmas, Thanksgiving, Labor day) and add that as extra weight to release date?
* Has rating and popularity index
* DOes not have top three actor names

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import sqlite3


In [26]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM movie_data
   """

# bring in db to pandas dataframe
movies = pd.read_sql(sql, con)
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [20]:
# # importing movie table
# movies = pd.read_csv("movie_metadata.csv")
# movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [ ]:
movies.columns
# Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
#        'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
#        'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
#        'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
#        'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
#        'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
#        'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
#        'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
#       dtype='object')

movies.dtypes


In [11]:
# importing oscar nominations and wins table
oscars = pd.read_csv("oscars.csv")
oscars.head()

,year,category,winner,entity
0,1927,ACTOR,False,Richard Barthelmess
1,1927,ACTOR,True,Emil Jannings
2,1927,ACTRESS,False,Louise Dresser
3,1927,ACTRESS,True,Janet Gaynor
4,1927,ACTRESS,False,Gloria Swanson


In [ ]:
# * Search each entry in df movies: 
#     - (1) save "title_year" (need to change this to int), 
#     - (2) save string within column actor_1_name, 
#     - (3) search df oscars with conditional statement: if year is < title_year and name is the same, add 1 to counter for number of Oscars (or count the number of entries), 
#     - (4) add new column to movies called "actor_1_oscar_nods" and enter the number from the counter
#     - (5) repeat for actor_2_name and actor_3_name
#     - (6) calculate the total noms
#     - (5) repeat for total wins

# * Note that this will count Oscar nominations and wins.  For wins only, set conditional statement to look for "True" under "winner" column
# Note that I can't calculate the oscar wins for the director, as oscar.csv records the directing oscars by movie title, not director


In [ ]:
# use iloc to pull the information on year and actor names
# release_year = int(movies.iloc[0, 23])
# actor1_name = movies.iloc[0, 10] # CCH Pounder
# actor2_name = movies.iloc[0, 6] # Joel David Moore
# actor3_name = movies.iloc[0, 14] # Wes Studi
# actor1_name
# release_year

In [ ]:
# change index to entity
oscars2 = oscars
oscars2 = oscars.set_index("entity")

test_name = oscars[oscars["entity"] == "Meryl Streep"] #new df of all occurances
test_count = oscars2.loc[["Meryl Streep"]].count()
test_nods = len(test_name[(test_name['year'] < release_year)]) #gives int; 15 for Meryl Streep when release year = 2009
test_wins = len(test_name[(test_name['year'] < release_year) & (test_name['winner'] == True)])
test_wins # 2 for Meryl Streep when release_year = 2009
test_nods



In [ ]:
#Note that the string has to be perfect with this syntaxt. So "Robert Downey Jr." will pull out his 2 Oscar noms
# but "Robert Downey Jr" will not:
test_name2 = oscars[oscars["entity"] == "Robert Downey Jr"] #new df of all occurances

# So instead, use str.contains(); regex = False means regex method isn't used when it's not needed, like in the case of a perfect match
test_name2 = oscars[oscars["entity"].str.contains("Robert Downey Jr", regex = False)] #new df of all occurances
test_name2


In [ ]:
# add new column to df and populate with 0
movies2 = movies
# movies["oscar_noms"] = 0
# movies["oscar_wins"] = 0
# movies.head()

# Add number of oscar nominations and wins to proper place in database
# movies.iloc[0, 28] = test_nods
# movies.iloc[0, 29] = test_wins
movies2.head(10)

movies3 = movies2.iloc[0:9]
movies3

In [ ]:
# need to replace NaN in title_year for code to work
movies3.fillna(0, inplace= True)
movies3["title_year"] = movies3["title_year"].astype(int)
# movies3[["title_year"]] = movies3[["title_year"]].fillna(value=0)
movies3

In [ ]:
# import math
# movies3['title_year'] = movies3['title_year'].apply(lambda x : x if math.isnan(x) else int(x)) 

In [ ]:
# Loop through rows of movies3 df to capture info and save date back to 
# test function on a small dataset, movies3 df
for i in range(len(movies4)) : 

    release_year = int(movies4.iloc[i, 23])
#     if movies.iloc[i, 23]:
#         release_year = int(movies.iloc[i, 23])
#     elif ValueError:
#         pass
#     else:
#         pass
#     release_year = movies.iloc[i, 23]
#     if release_year.isdigit():
#       release_year = int(release_year)
#     elif ValueError:
#         pass
#     try:
#         int(release_year)
#     except ValueError:
#         pass
    
    print(release_year)
    actor1_name = movies.iloc[i, 10] # CCH Pounder
    
    actor1_oscars = oscars[oscars["entity"] == actor1_name] #new df of all occurances of actor1's name
    actor1_nods = len(actor1_oscars[(actor1_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor1_wins = len(actor1_oscars[(actor1_oscars['year'] < release_year) & (actor1_oscars['winner'] == True)])

    actor2_name = movies.iloc[i, 6] # Joel David Moore
    actor2_oscars = oscars[oscars["entity"] == actor2_name] #new df of all occurances of actor1's name
    actor2_nods = len(actor2_oscars[(actor2_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor2_wins = len(actor2_oscars[(actor2_oscars['year'] < release_year) & (actor2_oscars['winner'] == True)])
    
    actor3_name = movies.iloc[i, 14] # Wes Studi
    actor3_oscars = oscars[oscars["entity"] == actor3_name] #new df of all occurances of actor1's name
    actor3_nods = len(actor3_oscars[(actor3_oscars['year'] < release_year)]) #gives int of # of appearances on list; i.e. total # of noms and wins
    actor3_wins = len(actor3_oscars[(actor3_oscars['year'] < release_year) & (actor3_oscars['winner'] == True)])

In [ ]:
movies3

In [ ]:
# def awards(row, actor_num, win=False):
#     global oscars
#     release_year = row['title_year']
#     actor1_name = row[f"actor_{str(actor_num)}_name"] # CCH Pounder
# #     actor2_name = movies3.iloc[0:, 6] # Joel David Moore
# #     actor3_name = movies3.iloc[0:, 14] # Wes Studi
    
#     noms = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) #gives int; 15 for Meryl Streep when release year = 2009
#     wins = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
#     return wins if win else noms



In [12]:
def award_noms(row):
    global oscars
    release_year = row['title_year']
    actor1_name = row["actor_1_name"] 
    actor2_name = row["actor_2_name"]  
    actor3_name = row["actor_3_name"]  
    
    noms1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    tot_noms = noms1 + noms2 + noms3
    tot_wins = wins1 + wins2 + wins3

    return tot_noms



In [ ]:
movies3["tot_noms"] = movies3.apply(award_noms, axis=1)

In [ ]:
movies3

In [13]:
def award_wins(row):
    global oscars
    release_year = row['title_year']
    actor1_name = row["actor_1_name"] 
    actor2_name = row["actor_2_name"]  
    actor3_name = row["actor_3_name"]  
    
    noms1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins1 = len(oscars[(oscars['entity'].str.contains(actor1_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins2 = len(oscars[(oscars['entity'].str.contains(actor2_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    noms3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year)]) 
    wins3 = len(oscars[(oscars['entity'].str.contains(actor3_name, regex = False)) & (oscars['year'] < release_year) & (oscars['winner'] == True)])
    
    tot_noms = noms1 + noms2 + noms3
    tot_wins = wins1 + wins2 + wins3

    return tot_wins


In [ ]:
movies3["tot_wins"] = movies3.apply(award_wins, axis=1)

In [ ]:
movies3

# Redo functions for entire movies dataset:

In [27]:
# edit original df to change NaN to zero and change float to int
movies2 = movies.copy(deep=True)
movies2.fillna(0, inplace= True)
movies2["title_year"] = movies2["title_year"].astype(int)
movies2.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [ ]:
# functions are the same, just need to call the movies2 df now to get data for all 5000 movies

In [28]:
movies2["tot_noms"] = movies2.apply(award_noms, axis=1)
movies2["tot_wins"] = movies2.apply(award_wins, axis=1)
movies2.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [29]:
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,release_date,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,2009-12-10,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,1998-01-30,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,2005-12-25,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,2006-03-09,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,2012-01-19,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0


In [30]:
movies["tot_noms"] = movies2["tot_noms"]
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,success_bins,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,extreme success,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,no success,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,no success,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,moderate success,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,average success,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0


In [31]:
movies["tot_wins"] = movies2["tot_wins"]

movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [32]:
# store dataframe in a sqlite db
con = sqlite3.connect("db/movies.db")
movies.to_sql("movie_data", con, if_exists="replace", index=False)
# commit the changes and close the connection
con.commit()
con.close()

In [33]:
# store dataframe in a sqlite db
con = sqlite3.connect("db/movies.db")
oscars.to_sql("oscars", con, if_exists="replace", index=False)
# commit the changes and close the connection
con.commit()
con.close()

In [34]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM movie_data
   """

# bring in db to pandas dataframe
movieeees = pd.read_sql(sql, con)
movieeees.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,production_companies_count,genres_count,popularity,vote_count,vote_average,total_actor_starpower,release_month,holiday_month,tot_noms,tot_wins
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,4.0,4,185.070892,12114.0,7.2,5.334665,12,1,0,0
1,Color,Stephen Sommers,106.0,106.0,208.0,855.0,Jason Flemyng,3000.0,11146409.0,Action|Adventure|Horror|Sci-Fi,...,3.0,4,6.922458,155.0,6.0,10.933174,01,0,0,0
2,Color,Terrence Malick,222.0,150.0,0.0,855.0,Michael Greyeyes,23000.0,12712093.0,Biography|Drama|History|Romance,...,5.0,3,7.694502,336.0,6.4,11.013046,12,1,0,0
3,Color,Brian Robbins,76.0,98.0,48.0,722.0,Joel David Moore,21000.0,61112916.0,Comedy|Family|Fantasy,...,3.0,2,4.878907,138.0,4.5,13.698882,03,0,1,0
4,Color,Brad Peyton,178.0,94.0,62.0,722.0,Dwayne Johnson,14000.0,103812241.0,Action|Adventure|Comedy|Family|Fantasy|Sci-Fi,...,3.0,3,9.46307,1050.0,5.8,6.475240,01,0,0,0


In [35]:
# create connection to database

con = sqlite3.connect("db/movies.db")
sql = f"""
   SELECT * FROM oscars
   """

# bring in db to pandas dataframe
oscarzz = pd.read_sql(sql, con)
oscarzz.head()

,year,category,winner,entity
0,1927,ACTOR,0,Richard Barthelmess
1,1927,ACTOR,1,Emil Jannings
2,1927,ACTRESS,0,Louise Dresser
3,1927,ACTRESS,1,Janet Gaynor
4,1927,ACTRESS,0,Gloria Swanson
